In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-23 23:05:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.75MB/s    in 0.2s    

2022-11-23 23:05:48 (5.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://sierrabucket1117.s3.amazonaws.com/amazon_reviews_us_Books_v1_02.tsv.gz" 
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [6]:
#Count Rows
books_df.count()

3105520

In [7]:
#Print Schema
books_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
#Drop Null Review_ID, Product ID, Customer ID

dropna_df = books_df.na.drop(subset=["review_id","product_id","customer_id"]) 
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [9]:
dropna_df.count()

3105520

In [10]:
# Load in a sql function to use columns
from pyspark.sql.functions import col


In [11]:
#Filter Unique review_ids


from pyspark.sql.functions import countDistinct
unique_review_df=dropna_df.select(countDistinct("review_id"))
unique_review_df.show()


+-------------------------+
|count(DISTINCT review_id)|
+-------------------------+
|                  3105520|
+-------------------------+



In [12]:
# Create user dataframe to match review_id_table table
books_review_id_df = dropna_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
books_review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979| 2005-10-14|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234| 2005-10-14|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049| 2005-10-14|
| RATOTLA3OF70O|   50732546|0373836635|     348672532| 2005-10-14|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341| 2005-10-14|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889| 2005-10-14|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054| 2005-10-14|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133| 2005-10-14|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124| 2005-10-14|
|R2UP6XSVYJBJ2H|   24209213|0976822105|     731931430| 2005-10-14|
|R21SYDQ70ILUC0|   52253037|1580085695|     586052746| 2005-10

In [13]:
# Create user dataframe to match products table
books_products_df = dropna_df.select(["product_id", "product_title"])
books_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0385730586|Sisterhood of the...|
|0811828964|The Bad Girl's Gu...|
|1844161560|Eisenhorn (A Warh...|
|0373836635|Colby Conspiracy ...|
|0262181533|The Psychology of...|
|0316769487|The Catcher in th...|
|0805076069|Bait and Switch: ...|
|1581603681|Opening Combinati...|
|0300108834|A Little History ...|
|0446531081|Hour Game (King &...|
|0976822105|     Faith is a Verb|
|1580085695|Furry Logic: A Gu...|
|0029148510|Acts of War: Beha...|
|1592285570|Temple to the Win...|
|0811848833|The Star Wars Pos...|
|0782144276|Photoshop for Nat...|
|0375757996|The Basic Works o...|
|013146308X|The Pterosaurs: F...|
|0375701907|Straight Man: A N...|
|141378240X|   Lipstick's Legacy|
+----------+--------------------+
only showing top 20 rows



In [14]:
#Filter Unique Customer IDs

unique_customer=dropna_df.select(countDistinct("customer_id"))
unique_customer.show()

+---------------------------+
|count(DISTINCT customer_id)|
+---------------------------+
|                    1502380|
+---------------------------+



In [15]:
# Create user dataframe to match customers table
books_customers_df = dropna_df.groupBy("customer_id").count()
books_customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   28735044|    9|
|   15607794|    1|
|   15596972|    1|
|   32371281|    2|
|   12084021|    1|
|   14462635|    1|
|   41052840|    5|
|   52068296|  105|
|   51739623|    1|
|   14544320|    1|
|   50093726|  151|
|   52925615|    1|
|   12266315|    1|
|   44917096|    1|
|   52833886|    1|
|   51854558|   57|
|   12404586|    2|
|   49850307|    8|
|   23478170|    1|
|   51088740|    1|
+-----------+-----+
only showing top 20 rows



In [16]:
# Rename Column for Customer Table 

books_customers_df.withColumnRenamed("count", "customer_count")


DataFrame[customer_id: int, customer_count: bigint]

In [17]:
# Create user dataframe to match vine table
books_vine_df = dropna_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
books_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
|R2UP6XSVYJBJ2H|          5|            0|          0|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
| RRS38KZ4WB5O2|          5|            

Postgres Setup

In [23]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}


In [24]:
# Write DataFrame to review_id table in RDS

books_review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [25]:
# Write dataframe to products table in RDS

books_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [26]:
# Write dataframe to customers  table in RDS

books_customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [27]:
# Write dataframe to vine table in RDS

books_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)